In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import commonmodules as cm

filename = "N2H2_VVdata_3variables.xlsx"
df = pd.read_excel(filename)
debug = False
print(df.columns)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

x = df[['v', 'w', 'T(K)']].values
y = df[['k(cm^3/s)']].values

scalerx = MinMaxScaler()
scalerx.fit(x)
x_s = scalerx.transform(x)

vset = set(x_s[:,0])
wset = set(x_s[:,1])
tset = set(x_s[:,2])

scalery = MinMaxScaler()
scalery.fit(y)
y_s = scalery.transform(y)

if debug:
    for i, ys in enumerate(y_s):
        print(ys, y[i])
    for i, xs in enumerate(x_s):
        print(xs, x[i])

In [ ]:
from tensorflow import keras
import tensorflow as tf

import tensorflow.keras.optimizers as tko
import tensorflow.keras.activations as tka
import tensorflow.keras.losses as tkl
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

from sklearn.model_selection import train_test_split

def test_train_split (column, valuestotest, x, y):
    
    xtest = []
    ytest = []
    xtrain = []
    ytrain = []
    
    for v in valuestotest:
        for i, xv in enumerate(x[:,column]):
            if xv == v:
                xtest.append(x[i,:])
                ytest.append(y[i])
            else:
                xtrain.append(x[i,:])
                ytrain.append(y[i])   

    return np.asarray(xtrain), np.asarray(xtest), \
        np.asarray(ytrain), np.asarray(ytest)

def buildmodel(modelshape):
    model = keras.Sequential()
    model.add(keras.layers.Dense(units = 3, activation = 'linear', input_shape=[3]))

    for n in modelshape:
        model.add(keras.layers.Dense(units = n, activation = 'relu'))

    model.add(keras.layers.Dense(units = 1, activation = 'linear'))
    model.compile(loss='mse', optimizer="adam", metrics='mse')

    return model

def plotfull3dcurve (columntorm, x, y):

    yv = []
    xv = []
    for i, v in enumerate(x):
        xv.append()
        yv.append() 

    X = np.array(xv)
    Y = np.array(yv)

    xdim = len(temp_values)
    ydim = len(vib_values)

    Xp = np.zeros((xdim, ydim), dtype=float)
    Yp = np.zeros((xdim, ydim), dtype=float)
    Zp = np.zeros((xdim, ydim), dtype=float)
    for xidx in range(xdim):
        t = temp_values[xidx]
        for yidx in range(ydim):
            v =  vib_values[yidx]
            Xp[xidx, yidx] = float(t)
            Yp[xidx, yidx] = float(v)
            Zp[xidx, yidx] = df[t].values[yidx]

    #fig = plt.figure(figsize=(10,8))
    fig = plt.figure(figsize=plt.figaspect(2.))
    plt.gcf().set_size_inches(40, 30)
    ax = fig.add_subplot(2,1,1, projection='3d')
    surf = ax.plot_surface(Xp, Yp, Zp, rstride=1, cstride=1, cmap='jet', linewidth=0, antialiased=False)
    plt.show()
def plot (x, y):

  cm.plotfull3dcurve (df, vib_values, temp_values)

  train_xy, train_z, test_xy, test_z = cm.get_train_and_test_rmv (temp_values, vib_values, \
    df, testsetvib)

  fig = plt.figure()
  ax = fig.add_subplot(projection='3d')

  for i in range(train_z.shape[0]):
    x = train_xy[i,0]
    y = train_xy[i,1]
    z = train_z[i]
    ax.scatter(x, y, z, marker="o", color="g")

  for i in range(test_z.shape[0]):
    x = test_xy[i,0]
    y = test_xy[i,1]
    z = test_z[i]
    ax.scatter(x, y, z, marker="o", color="r")

  ax.set_xlabel('X Label')
  ax.set_ylabel('Y Label')
  ax.set_zlabel('Z Label')
  plt.gcf().set_size_inches(20, 15)

  plt.show()

In [ ]:
# chek data
for w in wset:
    train_x, test_x, train_y, test_y = test_train_split (1, [w], x_s, y_s)

    

In [ ]:
from sklearn import metrics

print (" Perc. Split , Test MSE , Test R2 , Train MSE , Train R2")
for perc in [0.05, 0.10, 0.25, 0.30, 0.50]:
    train_x, test_x, train_y, test_y = train_test_split(x_s, y_s, \
                    test_size=perc, random_state=42)

    modelshape = [32, 32, 32, 32]
    epochs = 10
    batch_size = 50

    model = buildmodel(modelshape)
    history = model.fit(train_x, train_y, epochs=epochs,  batch_size=batch_size, \
        verbose=0)

    pred_y = model.predict(test_x, verbose=0)
    #to scale back y
    #pred_y_sb = scalery.inverse_transform(pred_y)
    #y_sb = scalery.inverse_transform(test_y)
    #plt.scatter(y_sb, pred_y_sb)
    #plt.show()
    testmse = metrics.mean_absolute_error(test_y, pred_y)
    testr2 = metrics.r2_score(test_y, pred_y)

    pred_y = model.predict(train_x, verbose=0)
    trainmse = metrics.mean_absolute_error(train_y, pred_y)
    trainr2 = metrics.r2_score(train_y, pred_y)

    print("%5.2f , %10.6f , %10.6f , %10.6f , %10.6f"%(perc, testmse, testr2, \
                                                       trainmse,  trainr2))

In [ ]:
print (" v Removed , Test MSE , Test R2 , Train MSE , Train R2")
for v in vset:
    train_x, test_x, train_y, test_y = test_train_split (0, [v], x_s, y_s)

    modelshape = [32, 32, 32, 32]
    epochs = 10
    batch_size = 50

    model = buildmodel(modelshape)
    history = model.fit(train_x, train_y, epochs=epochs,  batch_size=batch_size, \
        verbose=0)

    pred_y = model.predict(test_x, verbose=0)
    #to scale back y
    #pred_y_sb = scalery.inverse_transform(pred_y)
    #y_sb = scalery.inverse_transform(test_y)
    #plt.scatter(y_sb, pred_y_sb)
    #plt.show()
    testmse = metrics.mean_absolute_error(test_y, pred_y)
    testr2 = metrics.r2_score(test_y, pred_y)

    pred_y = model.predict(train_x, verbose=0)
    trainmse = metrics.mean_absolute_error(train_y, pred_y)
    trainr2 = metrics.r2_score(train_y, pred_y)

    print("%5.2f , %10.6f , %10.6f , %10.6f , %10.6f"%(v, testmse, testr2, \
                                                       trainmse,  trainr2))

In [ ]:
print (" w Removed , Test MSE , Test R2 , Train MSE , Train R2")
for w in wset:
    train_x, test_x, train_y, test_y = test_train_split (1, [w], x_s, y_s)

    modelshape = [32, 32, 32, 32]
    epochs = 10
    batch_size = 50

    model = buildmodel(modelshape)
    history = model.fit(train_x, train_y, epochs=epochs,  batch_size=batch_size, \
        verbose=0)

    pred_y = model.predict(test_x, verbose=0)
    #to scale back y
    pred_y_sb = scalery.inverse_transform(pred_y)
    y_sb = scalery.inverse_transform(test_y)
    plt.scatter(y_sb, pred_y_sb)
    plt.show()
    testmse = metrics.mean_absolute_error(test_y, pred_y)
    testr2 = metrics.r2_score(test_y, pred_y)

    pred_y = model.predict(train_x, verbose=0)
    trainmse = metrics.mean_absolute_error(train_y, pred_y)
    trainr2 = metrics.r2_score(train_y, pred_y)

    print("%5.2f , %10.6f , %10.6f , %10.6f , %10.6f"%(w, testmse, testr2, \
                                                       trainmse,  trainr2))

In [ ]:
print (" T Removed , Test MSE , Test R2 , Train MSE , Train R2")
for t in tset:
    train_x, test_x, train_y, test_y = test_train_split (2, [t], x_s, y_s)

    modelshape = [32, 32, 32, 32]
    epochs = 10
    batch_size = 50

    model = buildmodel(modelshape)
    history = model.fit(train_x, train_y, epochs=epochs,  batch_size=batch_size, \
        verbose=0)

    pred_y = model.predict(test_x, verbose=0)
    #to scale back y
    #pred_y_sb = scalery.inverse_transform(pred_y)
    #y_sb = scalery.inverse_transform(test_y)
    #plt.scatter(y_sb, pred_y_sb)
    #plt.show()
    testmse = metrics.mean_absolute_error(test_y, pred_y)
    testr2 = metrics.r2_score(test_y, pred_y)

    pred_y = model.predict(train_x, verbose=0)
    trainmse = metrics.mean_absolute_error(train_y, pred_y)
    trainr2 = metrics.r2_score(train_y, pred_y)

    print("%5.2f , %10.6f , %10.6f , %10.6f , %10.6f"%(t, testmse, testr2, \
                                                       trainmse,  trainr2))